In [1]:
# dependencies
import time

# helper function declearation & implementation
def displayTime(timestamp: float = None) -> str:
    if timestamp is None:
        timestamp = time.time()
        
    return f"{time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timestamp))}.{str(timestamp).split('.')[1][:3]}"

# Function Decorators in Python

*The current method for transforming functions and methods (for instance, declaring them as a class or static method) is awkward and can lead to code that is difficult to understand. Ideally, these transformations should be made at the same point in the code where the declaration itself is made. This PEP introduces new syntax for transformations of a function or method declaration.* - PEP 318


A decorator is a **design pattern** that allows a user to add new functionality to an existing object with out modifying its structure.

In python, functions are **first-class citizens**, meaning that a function can be: 
  * assigned to a variable
  * being passed as an argument
  * being returned by a function

Also, python allows us to define a function within a function.

In [2]:
def functionA():
    print("functionA is being called.")
    
someRandomName = functionA
someRandomName()

functionA is being called.


In [3]:
def functionB1():
    print("functionB1 is being called.")
    
def functionB2(function):
    print("functionB2 is being called.")
    function()
    
functionB2(functionB1)

functionB2 is being called.
functionB1 is being called.


In [4]:
def functionC1():
    print("functionC1 is being called.")
    
    def functionC2():
        print("functionC2 is being called.")
        
    print("functionC2 is not called... yet.")
    return functionC2

newFun = functionC1()
newFun()

functionC1 is being called.
functionC2 is not called... yet.
functionC2 is being called.


To add extra functionality to a existing function, or to *decorate* that function, our **decorator** could:
  * do something extra before the function is called
  * call the function, so it can do its own work
  * do something after the function had finished working

Thus, a decorator is structured like this: 

In [5]:
def decorate(function_to_decorate):
    def wrapper():
        # things done before the passed-in function is called
        print("This function is wrapped! --> ", end = '')
        # -------------------
        
        function_to_decorate()
        
        # things done after the passed-in function finished execution
        print("Everything is done! ")
        # --------------------

    return wrapper

In [6]:
def functionD1():
    print("functionD1 is being called.")
    
decoratedD1 = decorate(functionD1)
decoratedD1()

@decorate
def functionD2():
    print("functionD2 is being called.")
    
functionD2()

This function is wrapped! --> functionD1 is being called.
Everything is done! 
This function is wrapped! --> functionD2 is being called.
Everything is done! 


We can also pass arguments to the function we decorated, for example, this logger recognize what arguments are passed in to the function we called:

In [7]:
def logger(func):
    def loggedFunction(*args, **kwargs):
        
        # things done before the passed-in function is called
        start = time.perf_counter()
        print(f"[{displayTime()}] Function {func.__name__}() started with arguments {str(args), str(kwargs)}")
        # -------------------
        
        returnedValues = func(*args, **kwargs)
        
        # things done after the passed-in function finished execution
        end = time.perf_counter()
        print(f"[{displayTime()}] Function {func.__name__}() ended with return value {str(returnedValues)}")
        print(f"[{displayTime()}] Function execution took {(end - start)*1000:.3f} ms")
        # --------------------
        
        return returnedValues
    
    return loggedFunction

In [8]:
@logger
def myFunction1(n: int, *args):
    print("<function1 executing...>")
    time.sleep(n)
    print("<function1 exiting...>")
    return n

@logger
def myFunction2(n: int, **kwargs):
    print("<function1 executing...>")
    time.sleep(n)
    print("<function1 exiting...>")
    return n

myFunction1(1, 91, 12, 28)
myFunction2(2, kwarg1 = 16, kwarg2 = 63)

[2022-09-21 14:04:26.376] Function myFunction1() started with arguments ('(1, 91, 12, 28)', '{}')
<function1 executing...>
<function1 exiting...>
[2022-09-21 14:04:27.383] Function myFunction1() ended with return value 1
[2022-09-21 14:04:27.383] Function execution took 1006.390 ms
[2022-09-21 14:04:27.383] Function myFunction2() started with arguments ('(2,)', "{'kwarg1': 16, 'kwarg2': 63}")
<function1 executing...>
<function1 exiting...>
[2022-09-21 14:04:29.386] Function myFunction2() ended with return value 2
[2022-09-21 14:04:29.386] Function execution took 2002.616 ms


2

If we nest decorator inside another decorator, we can also pass arguments to the decorator inside!

For example, look at this advanced logger below:

In [9]:
def advanced_logger(alert: bool = False):
    def decorator(func):
        def wrapper(*args, **kwargs):
            
            # things done before the passed-in function is called
            start = time.perf_counter()
            if alert:
                print("\n[!----        ALERT         ----!]")
            print(f"[{displayTime()}] Function {func.__name__}() started with arguments {str(args), str(kwargs)}")
            # -------------------
            
            returnedValues = func(*args, **kwargs)
            
            # things done after the passed-in function finished execution
            end = time.perf_counter()
            print(f"[{displayTime()}] Function {func.__name__}() ended with return value {str(returnedValues)}")
            print(f"[{displayTime()}] Function execution took {(end - start)*1000:.3f} ms")
            if alert:
                print("[!---- ALERT SECTION ENDING ----!]\n")
            # --------------------
            
            return returnedValues
        
        return wrapper

    return decorator

In [10]:
@advanced_logger()
def normal_function():
    print("Hello, I am just a function.")

@advanced_logger(alert = True)
def dangerous_function():
    print("Hello, I am a dangerous function that needs to be alerted on execution!")
    
normal_function()
dangerous_function()
normal_function()

[2022-09-21 14:04:29.418] Function normal_function() started with arguments ('()', '{}')
Hello, I am just a function.
[2022-09-21 14:04:29.419] Function normal_function() ended with return value None
[2022-09-21 14:04:29.419] Function execution took 0.744 ms

[!----        ALERT         ----!]
[2022-09-21 14:04:29.419] Function dangerous_function() started with arguments ('()', '{}')
Hello, I am a dangerous function that needs to be alerted on execution!
[2022-09-21 14:04:29.420] Function dangerous_function() ended with return value None
[2022-09-21 14:04:29.420] Function execution took 0.520 ms
[!---- ALERT SECTION ENDING ----!]

[2022-09-21 14:04:29.420] Function normal_function() started with arguments ('()', '{}')
Hello, I am just a function.
[2022-09-21 14:04:29.421] Function normal_function() ended with return value None
[2022-09-21 14:04:29.421] Function execution took 0.331 ms


## References

https://peps.python.org/pep-0318/

https://www.datacamp.com/tutorial/decorators-python